In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
output_path = "/content/drive/MyDrive/DSP_pr/2510.18234v1_vi.md"


In [3]:
# Bước 1: Cài đặt thư viện cần thiết
!pip install transformers accelerate torch sentencepiece bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login(new_session=True)

In [4]:
# ============================================================
# CELL 1: LOAD MODEL (CHẠY 1 LẦN DUY NHẤT)
# ===========================================================
import warnings
warnings.filterwarnings('ignore')

# Import thư viện
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

# Load model HY-MT1.5-7B
print("=" * 60)

print("Lưu ý: Mô hình lớn, có thể mất 5-10 phút để tải")
print("=" * 60)

model_name = "tencent/HY-MT1.5-7B"
print("Đang tải mô hình" ,model_name,"...")
# Load với 4-bit quantization
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    trust_remote_code=True
)

print("✓ Model đã sẵn sàng!")
print(f"Device: {model.device}")
print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print("\n" + "=" * 60)
print("✅ CELL 1 HOÀN THÀNH - Model đã được load vào bộ nhớ")
print("Bây giờ bạn có thể chạy CELL 2 bao nhiêu lần tùy thích!")
print("=" * 60)


Lưu ý: Mô hình lớn, có thể mất 5-10 phút để tải
Đang tải mô hình tencent/HY-MT1.5-7B ...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

✓ Model đã sẵn sàng!
Device: cuda:0
Memory allocated: 4.66 GB

✅ CELL 1 HOÀN THÀNH - Model đã được load vào bộ nhớ
Bây giờ bạn có thể chạy CELL 2 bao nhiêu lần tùy thích!


In [8]:
# ============================================================
# CELL 2: DỊCH FILE + AUTOSAVE (CHẠY NHIỀU LẦN)
# ============================================================

import re
import torch
from google.colab import files

# ================= AUTOSAVE CONFIG =================
AUTOSAVE_EVERY = 5          # lưu sau mỗi 5 đoạn
AUTOSAVE_FILENAME = None    # sẽ set sau khi upload
# ===================================================

# =================================================================
# CẤU HÌNH GENERATION - TÙY CHỈNH THEO GPU
# =================================================================

GENERATION_CONFIG_A100 = {
    "max_new_tokens": 4096,
    "top_k": 20,
    "top_p": 0.6,
    "temperature": 0.7,
    "repetition_penalty": 1.05,
    "do_sample": True,
    "num_beams": 1,
}

GENERATION_CONFIG_T4 = {
    "max_new_tokens": 2048,
    "top_k": 20,
    "top_p": 0.6,
    "temperature": 0.7,
    "repetition_penalty": 1.05,
    "do_sample": True,
}

GENERATION_CONFIG_FREE = {
    "max_new_tokens": 4096,
    "top_k": 20,
    "top_p": 0.6,
    "temperature": 0.7,
    "repetition_penalty": 1.05,
    "do_sample": True,
}

# ===== CHỌN CẤU HÌNH =====
GENERATION_CONFIG = GENERATION_CONFIG_FREE

# ============================================================
# MARKDOWN HELPERS
# ============================================================

def extract_markdown_elements(text):
    elements = {
        'code_blocks': [],
        'inline_code': [],
        'latex_blocks': [],
        'latex_inline': [],
    }

    code_pattern = r'```[\s\S]*?```'
    for m in re.finditer(code_pattern, text):
        elements['code_blocks'].append(m.group(0))
    text = re.sub(code_pattern, lambda m: f'___CODE_BLOCK_{len(elements["code_blocks"])-1}___', text)

    latex_block_pattern = r'\$\$[\s\S]*?\$\$'
    for m in re.finditer(latex_block_pattern, text):
        elements['latex_blocks'].append(m.group(0))
    text = re.sub(latex_block_pattern, lambda m: f'___LATEX_BLOCK_{len(elements["latex_blocks"])-1}___', text)

    latex_inline_pattern = r'\$[^\$\n]+\$'
    for m in re.finditer(latex_inline_pattern, text):
        elements['latex_inline'].append(m.group(0))
    text = re.sub(latex_inline_pattern, lambda m: f'___LATEX_INLINE_{len(elements["latex_inline"])-1}___', text)

    inline_code_pattern = r'`[^`\n]+`'
    for m in re.finditer(inline_code_pattern, text):
        elements['inline_code'].append(m.group(0))
    text = re.sub(inline_code_pattern, lambda m: f'___INLINE_CODE_{len(elements["inline_code"])-1}___', text)

    return text, elements


def restore_markdown_elements(text, elements):
    for i, v in enumerate(elements['code_blocks']):
        text = text.replace(f'___CODE_BLOCK_{i}___', v)
    for i, v in enumerate(elements['latex_blocks']):
        text = text.replace(f'___LATEX_BLOCK_{i}___', v)
    for i, v in enumerate(elements['latex_inline']):
        text = text.replace(f'___LATEX_INLINE_{i}___', v)
    for i, v in enumerate(elements['inline_code']):
        text = text.replace(f'___INLINE_CODE_{i}___', v)
    return text


def split_text_smart(text, max_words=300):
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
    chunks, cur = [], ""
    for s in sentences:
        if len(cur.split()) + len(s.split()) < max_words:
            cur += s + " "
        else:
            if cur.strip():
                chunks.append(cur.strip())
            cur = s + " "
    if cur.strip():
        chunks.append(cur.strip())
    return chunks if chunks else [text]

# ============================================================
# TRANSLATION
# ============================================================

def translate_text(text):
    print(f"English text{text}")
    prompt = f"Translate the following segment into Vietnamese, without additional explanation (if some part have this format, keep it intact because it is the metadata <|ref|>text<|/ref|><|det|>[[x,y,z,t]]<|/det|>.Do not hold special token, only return output token).\n\n{text}"
    messages = [{"role": "user", "content": prompt}]

    tokenized = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=False,
        return_tensors="pt",
        return_token_type_ids=False
    )

    with torch.no_grad():
        outputs = model.generate(
            tokenized.to(model.device),
            **GENERATION_CONFIG,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if prompt in decoded:
        decoded = decoded.replace(prompt, "")
    decoded = decoded.replace("<|eos|>", "").strip()
    return decoded


def translate_markdown(markdown_text):
    paragraphs = markdown_text.split('\n\n')
    translated = []
    total = len(paragraphs)

    print(f"\n{'='*60}")
    print(f"Tổng số đoạn cần dịch: {total}")
    print(f"{'='*60}\n")

    for idx, para in enumerate(paragraphs):
        print(f"[{idx+1}/{total}]", end='')

        if not para.strip():
            translated.append(para)
            print(" ⏭️")
            continue

        if para.strip().startswith('#'):
            m = re.match(r'^(#+)\s+(.+)$', para.strip())
            if m:
                translated.append(f"{m.group(1)} {translate_text(m.group(2))}")
                print(" ✓ Header")
                continue

        processed, elements = extract_markdown_elements(para)
        chunks = split_text_smart(processed)
        out_chunks = []

        for c in chunks:
            try:
                out_chunks.append(translate_text(c))
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            except Exception as e:
                print(" ❌", e)
                out_chunks.append(c)

        final_para = restore_markdown_elements(' '.join(out_chunks), elements)
        translated.append(final_para)
        print(" ✓")

        # ================= AUTOSAVE =================
        if AUTOSAVE_FILENAME and (idx + 1) % AUTOSAVE_EVERY == 0:
            with open(AUTOSAVE_FILENAME, 'w', encoding='utf-8') as f:
                f.write('\n\n'.join(translated))
            print(f" 💾 Autosaved ({idx+1}/{total})")
        # ============================================

    return '\n\n'.join(translated)

# ============================================================
# UPLOAD FILE
# ============================================================

print("\n" + "="*60)
print("UPLOAD FILE MARKDOWN")
print("="*60)
uploaded = files.upload()

filename = list(uploaded.keys())[0]
AUTOSAVE_FILENAME = filename.replace('.md', '_autosave_vi.md')
print(f"📌 Autosave file: {AUTOSAVE_FILENAME}")

with open(filename, 'r', encoding='utf-8') as f:
    original_text = f.read()

# ============================================================
# TRANSLATE
# ============================================================

translated_text = translate_markdown(original_text)

# ============================================================
# SAVE FINAL FILES
# ============================================================

output_filename = filename.replace('.md', '_vi.md')
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(translated_text)

bilingual_filename = filename.replace('.md', '_bilingual.md')
with open(bilingual_filename, 'w', encoding='utf-8') as f:
    f.write("# Bản dịch song ngữ\n\n")
    for o, t in zip(original_text.split('\n\n'), translated_text.split('\n\n')):
        f.write(f"**English:** {o}\n\n**Tiếng Việt:** {t}\n\n---\n\n")

files.download(output_filename)
files.download(bilingual_filename)

print("\n🎉 HOÀN THÀNH!")



UPLOAD FILE MARKDOWN


Saving HY-MT1.5-7B.md to HY-MT1.5-7B (3).md
📌 Autosave file: HY-MT1.5-7B (3)_autosave_vi.md

Tổng số đoạn cần dịch: 185

[1/185] ⏭️
[2/185]English text============================================================
--- Page 1 ---
 ✓
[3/185]English text<|ref|>title<|/ref|><|det|>[[205, 132, 790, 157]]<|/det|>
# DeepSeek-OCR: Contexts Optical Compression
 ✓
[4/185]English text<|ref|>text<|/ref|><|det|>[[350, 187, 646, 203]]<|/det|>
Haoran Wei, Yaofeng Sun, Yukun Li
 ✓
[5/185]English text<|ref|>text<|/ref|><|det|>[[444, 215, 551, 230]]<|/det|>
DeepSeek- AI
 ✓
 💾 Autosaved (5/185)
[6/185]English text<|ref|>sub_title<|/ref|><|det|>[[450, 263, 547, 283]]<|/det|>
## Abstract
 ✓
[7/185]English text<|ref|>text<|/ref|><|det|>[[115, 306, 883, 564]]<|/det|>
We present DeepSeek- OCR as an initial investigation into the feasibility of compressing long contexts via optical 2D mapping. DeepSeek- OCR consists of two components: DeepEncoder and DeepSeek3B- MoE- A570M as the decoder. Specifically, DeepEncod

KeyboardInterrupt: 